### Notebook to figure out whats going on with the thresholder/corrector (doubling Z)

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import tables as tb
from matplotlib import colors 

from typing          import Optional
from typing          import Union
from typing          import Callable

from concurrent.futures import ProcessPoolExecutor

import sys,os,os.path
sys.path.append("/gluster/data/next/software/IC_311024/")
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))

#sys.path.append("/home/e78368jw/Documents/NEXT_CODE/next_misc/")
os.environ['ICTDIR']='/gluster/data/next/software/IC_311024/'


sys.path.append('/gluster/data/next/notebooks/john_books/sanity_booklet') 

from invisible_cities.io.dst_io           import load_dst, load_dsts, df_writer
from invisible_cities.io.hits_io          import hits_writer
from invisible_cities.core                import tbl_functions   as tbl
from invisible_cities.core.core_functions import in_range
from invisible_cities.cities.beersheba    import hitc_to_df_
from invisible_cities.io.hits_io          import hits_from_df
from invisible_cities.evm.nh5             import HitsTable
from invisible_cities.types.symbols       import NormStrategy
from invisible_cities.types.ic_types      import NoneType
from invisible_cities.reco.corrections    import read_maps, get_df_to_z_converter, apply_all_correction
from invisible_cities.evm.event_model     import HitCollection


from IC.invisible_cities.evm.event_model        import HitEnergy
from IC.invisible_cities.cities.beersheba          import DeconvolutionMode
from IC.invisible_cities.cities.beersheba          import CutType
from IC.invisible_cities.cities                 import beersheba as beerfun

import IC.invisible_cities.core.core_functions                   as     coref

from IC.invisible_cities.reco import hits_functions as hif

from IC.invisible_cities.reco.deconv_functions import deconvolve
from IC.invisible_cities.reco.deconv_functions import deconvolution_input
from IC.invisible_cities.reco.deconv_functions import InterpolationMethod


from tqdm import tqdm
import HE_calibration.functions_HE as func





In [9]:
RUN_NUMBER      = '15107'
TIMESTAMP_ISAU  = '240425'
TIMESTAMP_ESME  = '070525'

In [12]:
def load_file_soph_hits(file_path):
    '''
    Load data from a single h5 file and produce dataframes for /RECO/Events

    Args:
        file_path       :       str
                                Path to the h5 file to be loaded.

    Returns:
        tracks_df       :       pandas.DataFrame
                                DataFrame containing the /RECO/Events data.
    '''
    try: 
        tracks_df = load_dst(file_path, 'RECO', 'Events')
        return tracks_df
    except Exception as e:
        print(f'File {file_path} broke with error:\n{e}')
        x = pd.DataFrame()
        return x

In [13]:
n100_dir_soph = f'/gluster/data/next/files/TOPOLOGY_John/N100_data/{RUN_NUMBER}/sophronia_rebin/trigger_2/'

folder_path = n100_dir_soph + 'ldc1' + '/'

file_names = [f for f in os.listdir(folder_path) if f.endswith('.h5')]
soph_hits = load_file_soph_hits(f'{folder_path}{file_names[1]}')

In [18]:
display(soph_hits)

# take the first event
event = soph_hits[soph_hits.event == (soph_hits.event.unique()[0])]
print(f'Event: {event.event.unique()}\nZ average: {event.Z.mean():.2f} mm')

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,2177989,1.742963e+09,1,123.211798,115.277533,1,-389.325,-16.925,0.0,0.0,344.501188,5.136513,-1.068646,-1.0,-0.000003,-1,-1.0
1,2177989,1.742963e+09,1,123.211798,115.277533,1,42.075,446.075,0.0,0.0,344.501188,7.968457,-1.657829,-1.0,-0.000005,-1,-1.0
2,2177989,1.742963e+09,1,123.211798,115.277533,1,243.225,260.475,0.0,0.0,344.501188,6.052189,-1.259151,-1.0,-0.000004,-1,-1.0
3,2177989,1.742963e+09,1,123.211798,115.277533,1,396.725,-32.475,0.0,0.0,346.460500,5.087072,-0.822379,-1.0,-0.000002,-1,-1.0
4,2177989,1.742963e+09,1,123.211798,115.277533,1,412.275,106.475,0.0,0.0,346.460500,5.781449,-0.934632,-1.0,-0.000003,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6884,2178199,1.742963e+09,52,5.381660,61.311573,1,196.575,353.775,0.0,0.0,1765.128000,6.680068,-1.674934,-1.0,-0.000006,-1,-1.0
6885,2178199,1.742963e+09,52,5.381660,61.311573,1,351.075,260.975,0.0,0.0,1765.128000,6.415120,-1.608502,-1.0,-0.000006,-1,-1.0
6886,2178199,1.742963e+09,52,5.381660,61.311573,1,-420.425,260.975,0.0,0.0,1769.045000,5.157298,-1.271878,-1.0,NaN,-1,-1.0
6887,2178199,1.742963e+09,52,5.381660,61.311573,1,-312.575,-155.875,0.0,0.0,1769.045000,5.021692,-1.238436,-1.0,-0.000004,-1,-1.0


Event: [2177989]
Z average: 1255.90 mm


# get the corrector in here

In [29]:
from IC.invisible_cities.cities.components import hits_corrector
from IC.invisible_cities.io.hits_io import hits_from_df
from IC.invisible_cities.cities.beersheba import hitc_to_df_

In [46]:
correction_file = '/gluster/data/next/files/TOPOLOGY_John/LPR/LightTables/map_4bar_15063.h5'
apply_temp = False
correct_hits = hits_corrector(correction_file, apply_temp)

In [ ]:
# convert the event from df to hit collection
event_hitc = hits_from_df(event)
print(event_hitc)

{2177989: HitCollectionHit list:<Hit : npeak = 1.0 z = 344.5011875 XYpeak = 123.21179823724701, 115.27753290298035 E = -1.0686458408075854 Ec = -3.0703989643040945e-06 Ep = -1.0 trackid = -1.0 cluster =< nsipm = 1.0 Q = 5.136512756347656
                    xy = xy(x=-389.325, y=-16.925) 3dHit = Hit(-389.325, -16.925, 344.5011875, E=-1.0686458408075854)  > ><Hit : npeak = 1.0 z = 344.5011875 XYpeak = 123.21179823724701, 115.27753290298035 E = -1.657828679162515 Ec = -5.11860493564477e-06 Ep = -1.0 trackid = -1.0 cluster =< nsipm = 1.0 Q = 7.968456745147705
                    xy = xy(x=42.075, y=446.075) 3dHit = Hit(42.075, 446.075, 344.5011875, E=-1.657828679162515)  > ><Hit : npeak = 1.0 z = 344.5011875 XYpeak = 123.21179823724701, 115.27753290298035 E = -1.2591512518355437 Ec = -3.56930383021357e-06 Ep = -1.0 trackid = -1.0 cluster =< nsipm = 1.0 Q = 6.052188873291016
                    xy = xy(x=243.225, y=260.475) 3dHit = Hit(243.225, 260.475, 344.5011875, E=-1.2591512518355437) 

In [28]:
# correct hits
corrected_hitc = correct_hits(event_hitc[2177989])
print(corrected_hitc)

HitCollectionHit list:<Hit : npeak = 1.0 z = 749.4587597522288 XYpeak = 123.21179823724701, 115.27753290298035 E = -1.0686458408075854 Ec = -5.818417765600056e-06 Ep = -1.0 trackid = -1.0 cluster =< nsipm = 1.0 Q = 5.136512756347656
                    xy = xy(x=-389.325, y=-16.925) 3dHit = Hit(-389.325, -16.925, 749.4587597522288, E=-1.0686458408075854)  > ><Hit : npeak = 1.0 z = 749.4587597522288 XYpeak = 123.21179823724701, 115.27753290298035 E = -1.657828679162515 Ec = -1.0289142818139035e-05 Ep = -1.0 trackid = -1.0 cluster =< nsipm = 1.0 Q = 7.968456745147705
                    xy = xy(x=42.075, y=446.075) 3dHit = Hit(42.075, 446.075, 749.4587597522288, E=-1.657828679162515)  > ><Hit : npeak = 1.0 z = 749.4587597522288 XYpeak = 123.21179823724701, 115.27753290298035 E = -1.2591512518355437 Ec = -6.831991790205021e-06 Ep = -1.0 trackid = -1.0 cluster =< nsipm = 1.0 Q = 6.052188873291016
                    xy = xy(x=243.225, y=260.475) 3dHit = Hit(243.225, 260.475, 749.4587597522

In [30]:
# re-df em
corrected_df = hitc_to_df_(corrected_hitc)
display(corrected_df)

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,2177989,1.742963e+09,1.0,123.211798,115.277533,1.0,-389.325,-16.925,0.0,0.0,749.458760,5.136513,-1.068646,-1.0,-0.000006,-1.0,-1.0
1,2177989,1.742963e+09,1.0,123.211798,115.277533,1.0,42.075,446.075,0.0,0.0,749.458760,7.968457,-1.657829,-1.0,-0.000010,-1.0,-1.0
2,2177989,1.742963e+09,1.0,123.211798,115.277533,1.0,243.225,260.475,0.0,0.0,749.458760,6.052189,-1.259151,-1.0,-0.000007,-1.0,-1.0
3,2177989,1.742963e+09,1.0,123.211798,115.277533,1.0,396.725,-32.475,0.0,0.0,753.721224,5.087072,-0.822379,-1.0,-0.000004,-1.0,-1.0
4,2177989,1.742963e+09,1.0,123.211798,115.277533,1.0,412.275,106.475,0.0,0.0,753.721224,5.781449,-0.934632,-1.0,-0.000005,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,2177989,1.742963e+09,29.0,11.839304,38.297278,1.0,-142.525,369.325,0.0,0.0,5220.071587,7.950599,-0.792346,-1.0,-0.000005,-1.0,-1.0
1682,2177989,1.742963e+09,29.0,11.839304,38.297278,1.0,-34.675,291.575,0.0,0.0,5220.071587,5.049959,-0.503272,-1.0,-0.000003,-1.0,-1.0
1683,2177989,1.742963e+09,29.0,11.839304,38.297278,1.0,88.725,291.575,0.0,0.0,5220.071587,7.113851,-0.708957,-1.0,-0.000004,-1.0,-1.0
1684,2177989,1.742963e+09,29.0,11.839304,38.297278,1.0,165.475,-309.875,0.0,0.0,5220.071587,5.191612,-0.517389,-1.0,-0.000003,-1.0,-1.0


In [31]:
print(f'Event: {event.event.unique()}\nZ average: {event.Z.mean():.2f} mm')
print(f'Corrected Event: {corrected_df.event.unique()}\nZ average: {corrected_df.Z.mean():.2f} mm')

Event: [2177989]
Z average: 1255.90 mm
Corrected Event: [2177989]
Z average: 2732.19 mm


In [ ]:
#try a different correction map

# strip the corrector function and place in here

In [48]:


def hits_corrector_(map_fname  : str, 
                    apply_temp : bool,
                    apply_z    : Optional[bool] = False) -> Callable:
    """
    Applies energy correction map and converts drift time to z.

    Parameters
    ----------
    map_fname  : string (filepath)
        filename of the map
    apply_temp : bool
        whether to apply temporal corrections
        must be set to False if no temporal correction dataframe exists in map file

    Returns
    ----------
    A function that takes a HitCollection as input and returns
    the same object with modified Ec and Z fields.
    """

    map_fname = os.path.expandvars(map_fname)
    maps      = read_maps(map_fname)
    get_coef  = apply_all_correction(maps, apply_temp = apply_temp, norm_strat = NormStrategy.kr)
    time_to_Z = (get_df_to_z_converter_(maps) if maps.t_evol is not None and apply_z == True else
                 lambda x: x)

    def correct(hitc : HitCollection) -> HitCollection:
        for hit in hitc.hits:
            corr    = get_coef([hit.X], [hit.Y], [hit.Z], hitc.time)[0]
            print(f'hit {hit}')
            print(f'coefficient{corr}')
            print(f'Z and time_to_Z')
            print(f'{hit.Z}\n{time_to_Z(hit.Z)}')
            hit.Ec  = hit.E * corr
            hit.xyz = (hit.X, hit.Y, time_to_Z(hit.Z)) # ugly, but temporary
        return hitc

    return correct

In [49]:
correction_file = '/gluster/data/next/files/TOPOLOGY_John/LPR/LightTables/map_4bar_15063.h5'
apply_temp = False
correct_hits_ = hits_corrector_(correction_file, apply_temp)

In [50]:
corr_map = pd.read_hdf(correction_file, 'time_evolution')
display(corr_map)

#corr_file_gonzalo = '/gluster/data/next/files/TOPOLOGY_John/LPR/LightTables/rough_geometry_map_v0.h5'
#corr_map_gonzalo = pd.read_hdf(corr_file_gonzalo, 'time_evolution')
#display(corr_map_gonzalo)

,ts,e0,e0u,lt,ltu,dv,dvu,resol,resolu,s1w,...,s2qu,Nsipm,Nsipmu,Xrms,Xrmsu,Yrms,Yrmsu,S1eff,S2eff,Bandeff
0,1.742736e+09,8658.213284,inf,-8.242162e+10,inf,NaN,NaN,4.544419,0.031484,217.310117,...,2.803659,14.186259,0.035733,13.994111,0.028145,13.933625,0.028547,0.420532,0.949842,0.829609
1,1.742743e+09,8661.938664,4.606868,-3.184083e+11,8.392776e+16,2.091298,9.987170e+01,4.500554,0.054068,219.360104,...,2.865918,14.160627,0.037168,14.018331,0.029253,13.975092,0.028742,0.421161,0.947706,0.827367
2,1.742751e+09,8654.717004,inf,-3.187548e+11,inf,2.496811,3.084721e+00,4.499128,0.041360,215.650080,...,2.844632,14.117175,0.036569,14.014847,0.028600,13.930682,0.028349,0.419161,0.947624,0.826285
3,1.742759e+09,8655.736525,4.789883,-1.448498e+11,1.320228e+16,NaN,NaN,4.542072,0.028353,215.423702,...,2.815267,14.124140,0.035410,14.000486,0.027852,13.886305,0.028109,0.422475,0.947709,0.827401
4,1.742767e+09,8662.281285,4.782652,-1.463505e+11,1.418642e+16,NaN,NaN,4.500886,0.040988,217.323087,...,2.820639,14.136728,0.036265,14.013299,0.028941,13.957463,0.028487,0.421370,0.948182,0.826727
5,1.742775e+09,8651.652299,4.982505,-3.050135e+11,3.867849e+16,NaN,NaN,4.606824,0.069739,218.288274,...,2.822972,14.147124,0.036125,14.032190,0.028439,13.950554,0.027702,0.419516,0.948947,0.826710
6,1.742783e+09,8646.875789,inf,-6.456129e+11,inf,2.104283,1.790084e+03,4.570870,0.046200,216.908552,...,2.812257,14.125156,0.035903,14.007365,0.027712,13.912636,0.028054,0.423221,0.948548,0.827005
7,1.742791e+09,8728.005477,10.378030,3.424619e+04,4.086187e+03,2.157630,1.730692e+12,4.598146,0.039931,215.679191,...,2.844852,14.083494,0.036685,13.981919,0.028630,13.929352,0.028242,0.421278,0.948111,0.827233
8,1.742799e+09,8647.806444,5.328327,-1.127935e+11,1.001709e+16,2.083622,1.333465e+04,4.443942,0.039273,217.215617,...,2.841715,14.132069,0.036208,13.983769,0.028583,13.873304,0.027770,0.421917,0.947287,0.823754
9,1.742807e+09,8661.225956,inf,-6.876701e+11,inf,2.119293,4.177603e-02,4.453155,0.081394,218.908462,...,2.822219,14.182417,0.036821,14.042431,0.028587,14.010489,0.029056,0.424344,0.946497,0.823942


In [51]:
event_hitc = hits_from_df(event)
corrected_hitc = correct_hits_(event_hitc[2177989])


hit <Hit : npeak = 1.0 z = 344.5011875 XYpeak = 123.21179823724701, 115.27753290298035 E = -1.0686458408075854 Ec = -3.0703989643040945e-06 Ep = -1.0 trackid = -1.0 cluster =< nsipm = 1.0 Q = 5.136512756347656
                    xy = xy(x=-389.325, y=-16.925) 3dHit = Hit(-389.325, -16.925, 344.5011875, E=-1.0686458408075854)  > >
coefficient5.444664212797595e-06
Z and time_to_Z
344.5011875
344.5011875
hit <Hit : npeak = 1.0 z = 344.5011875 XYpeak = 123.21179823724701, 115.27753290298035 E = -1.657828679162515 Ec = -5.11860493564477e-06 Ep = -1.0 trackid = -1.0 cluster =< nsipm = 1.0 Q = 7.968456745147705
                    xy = xy(x=42.075, y=446.075) 3dHit = Hit(42.075, 446.075, 344.5011875, E=-1.657828679162515)  > >
coefficient6.206396925969938e-06
Z and time_to_Z
344.5011875
344.5011875
hit <Hit : npeak = 1.0 z = 344.5011875 XYpeak = 123.21179823724701, 115.27753290298035 E = -1.2591512518355437 Ec = -3.56930383021357e-06 Ep = -1.0 trackid = -1.0 cluster =< nsipm = 1.0 Q = 6.0521

# the final test, checking the z position of the new run

In [52]:
def load_file_beer_hits(file_path):
    '''
    Load data from a single h5 file and produce dataframes for /DECO/Events

    Args:
        file_path       :       str
                                Path to the h5 file to be loaded.

    Returns:
        tracks_df       :       pandas.DataFrame
                                DataFrame containing the /DECO/Events data.
    '''
    try: 
        tracks_df = load_dst(file_path, 'DECO', 'Events')
        return tracks_df
    except Exception as e:
        print(f'File {file_path} broke with error:\n{e}')
        x = pd.DataFrame()
        return x

In [78]:
n100_dir_soph = f'/gluster/data/next/files/TOPOLOGY_John/N100_data/{RUN_NUMBER}/sophronia_rebin/trigger_2/'

folder_path = n100_dir_soph + 'ldc1' + '/'

file_names = [f for f in os.listdir(folder_path) if f.endswith('.h5')]
soph_hits = load_file_soph_hits(f'{folder_path}run_15107_0001_ldc1_trg2.v2.3.1.20250328.HEDesmanTest.sophronia.h5')

In [79]:
n100_dir_beer = f'/gluster/data/next/files/TOPOLOGY_John/N100_data/15107/beersheba/130525/ldc1/run_15107_0001_ldc1_130525.h5'
beer_hits = load_file_beer_hits(f'{n100_dir_beer}')

In [ ]:
display(soph_hits)
display(beer_hits)
evt = 1353

evt_soph = soph_hits[soph_hits.event == evt]
evt_beer = beer_hits[beer_hits.event == evt]

display(evt_soph)
display(evt_beer)

print(f'Sophronia Z average: {evt_soph.Z.mean()}')
print(f'Beersheba Z average: {evt_beer.Z.mean()}')

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,842,1.742935e+09,0,7.063862,24.899767,1,-389.325,-62.575,0.0,0.0,-1041.928857,5.035276,-1.018071,-1.0,-0.000003,-1,-1.0
1,842,1.742935e+09,0,7.063862,24.899767,1,-126.975,307.125,0.0,0.0,-1041.928857,6.328022,-1.279448,-1.0,-0.000003,-1,-1.0
2,842,1.742935e+09,0,7.063862,24.899767,1,-126.975,492.725,0.0,0.0,-1041.928857,5.025098,-1.016013,-1.0,NaN,-1,-1.0
3,842,1.742935e+09,0,7.063862,24.899767,1,-265.925,-109.225,0.0,0.0,-1038.134307,5.050038,0.861560,-1.0,0.000002,-1,-1.0
4,842,1.742935e+09,0,7.063862,24.899767,1,-158.075,-464.375,0.0,0.0,-1038.134307,5.113098,0.872318,-1.0,NaN,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8322,1353,1.742935e+09,51,2.412014,120.910671,1,490.025,214.325,0.0,0.0,2773.530611,6.029944,-1.324141,-1.0,NaN,-1,-1.0
8323,1353,1.742935e+09,51,2.412014,120.910671,1,-359.225,75.375,0.0,0.0,2776.429361,5.706514,-0.297952,-1.0,-0.000001,-1,-1.0
8324,1353,1.742935e+09,51,2.412014,120.910671,1,-250.375,368.825,0.0,0.0,2776.429361,8.863242,-0.462773,-1.0,-0.000002,-1,-1.0
8325,1353,1.742935e+09,51,2.412014,120.910671,1,149.925,-78.625,0.0,0.0,2776.429361,6.734375,-0.351619,-1.0,-0.000001,-1,-1.0


,event,npeak,X,Y,Z,E,Xpeak,Ypeak,time,nsipm,Xrms,Yrms
0,842,33.0,71.125,196.325,729.859638,0.000097,42.126927,103.311974,1.742935e+09,0,0,0
1,842,33.0,71.125,198.825,729.859638,0.000122,42.126927,103.311974,1.742935e+09,0,0,0
2,842,33.0,71.125,201.325,729.859638,0.000142,42.126927,103.311974,1.742935e+09,0,0,0
3,842,33.0,71.125,203.825,729.859638,0.000153,42.126927,103.311974,1.742935e+09,0,0,0
4,842,33.0,71.125,206.325,729.859638,0.000152,42.126927,103.311974,1.742935e+09,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
18096,1353,31.0,371.125,201.325,3461.635527,0.000040,229.303563,142.810198,1.742935e+09,0,0,0
18097,1353,31.0,371.125,203.825,3461.635527,0.000024,229.303563,142.810198,1.742935e+09,0,0,0
18098,1353,31.0,373.625,196.325,3461.635527,0.000014,229.303563,142.810198,1.742935e+09,0,0,0
18099,1353,31.0,373.625,198.825,3461.635527,0.000019,229.303563,142.810198,1.742935e+09,0,0,0


,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,842,1.742935e+09,0,7.063862,24.899767,1,-389.325,-62.575,0.0,0.0,-1041.928857,5.035276,-1.018071,-1.0,-0.000003,-1,-1.0
1,842,1.742935e+09,0,7.063862,24.899767,1,-126.975,307.125,0.0,0.0,-1041.928857,6.328022,-1.279448,-1.0,-0.000003,-1,-1.0
2,842,1.742935e+09,0,7.063862,24.899767,1,-126.975,492.725,0.0,0.0,-1041.928857,5.025098,-1.016013,-1.0,NaN,-1,-1.0
3,842,1.742935e+09,0,7.063862,24.899767,1,-265.925,-109.225,0.0,0.0,-1038.134307,5.050038,0.861560,-1.0,0.000002,-1,-1.0
4,842,1.742935e+09,0,7.063862,24.899767,1,-158.075,-464.375,0.0,0.0,-1038.134307,5.113098,0.872318,-1.0,NaN,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,842,1.742935e+09,42,32.250213,-4.165335,1,-482.625,-294.825,0.0,0.0,1562.037500,5.033214,-1.144141,-1.0,NaN,-1,-1.0
1452,842,1.742935e+09,42,32.250213,-4.165335,1,-467.075,75.375,0.0,0.0,1562.037500,5.695229,-1.294629,-1.0,-0.000005,-1,-1.0
1453,842,1.742935e+09,42,32.250213,-4.165335,1,-204.725,-417.725,0.0,0.0,1562.037500,5.067519,-1.151939,-1.0,-0.000004,-1,-1.0
1454,842,1.742935e+09,42,32.250213,-4.165335,1,196.575,-294.325,0.0,0.0,1565.512250,5.153929,-3.471130,-1.0,-0.000011,-1,-1.0


,event,npeak,X,Y,Z,E,Xpeak,Ypeak,time,nsipm,Xrms,Yrms
0,842,33.0,71.125,196.325,729.859638,0.000097,42.126927,103.311974,1.742935e+09,0,0,0
1,842,33.0,71.125,198.825,729.859638,0.000122,42.126927,103.311974,1.742935e+09,0,0,0
2,842,33.0,71.125,201.325,729.859638,0.000142,42.126927,103.311974,1.742935e+09,0,0,0
3,842,33.0,71.125,203.825,729.859638,0.000153,42.126927,103.311974,1.742935e+09,0,0,0
4,842,33.0,71.125,206.325,729.859638,0.000152,42.126927,103.311974,1.742935e+09,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
625,842,33.0,86.125,188.825,771.055797,0.000031,42.126927,103.311974,1.742935e+09,0,0,0
626,842,33.0,86.125,191.325,771.055797,0.000048,42.126927,103.311974,1.742935e+09,0,0,0
627,842,33.0,86.125,193.825,771.055797,0.000056,42.126927,103.311974,1.742935e+09,0,0,0
628,842,33.0,86.125,196.325,771.055797,0.000049,42.126927,103.311974,1.742935e+09,0,0,0


Sophronia Z average: -0.20987301736346164
Beersheba Z average: 748.413616982523
